In [22]:
import mlflow
import pandas as pd 
import numpy as np
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error


In [15]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/654755040413823324', creation_time=1759177576069, experiment_id='654755040413823324', last_update_time=1759177576069, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [16]:
def read_dataframe(filename):
   
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [17]:
df_train = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-07.parquet")
df_val = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-08.parquet")

In [18]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
target = "duration"
y_train = df_train[target].values 
y_val = df_val[target].values

In [21]:
import xgboost as xgb

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val,label=y_val)


In [ ]:
mlflow.xgboost.autolog(disable=True)
with mlflow.start_run():
    
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label = y_val)
    
    best_params = {'learning_rate' : 0.10638129396766673,
                    'max_depth' : 29,
                    'min_child_weight': 5.759957543179141,
                    'objective' : 'reg:linear',
                    'reg_alpha' : 0.01851315389500763,
                    'reg_lambda' : 0.003994721106614769,
                    'seed' : 42}
    
    
    mlflow.log_params(best_params)
    
    booster = xgb.train(
    params=best_params, 
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50) 
    
    
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse) 
    
    with open("models/preprocessor.b", "wb") as f_out:
        
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor.b")
    mlflow.xgboost.log_model(booster,artifact_path="models_mlflow")


c:\Users\heysr\anaconda3\envs\projects\lib\site-packages\xgboost\core.py:158: UserWarning: [02:06:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.59376
[1]	validation-rmse:8.91276
[2]	validation-rmse:8.33341
[3]	validation-rmse:7.84360
[4]	validation-rmse:7.43323
[5]	validation-rmse:7.09107
[6]	validation-rmse:6.80745
[7]	validation-rmse:6.57595
[8]	validation-rmse:6.38664
[9]	validation-rmse:6.23415
[10]	validation-rmse:6.11042
[11]	validation-rmse:6.01230
[12]	validation-rmse:5.93374
[13]	validation-rmse:5.87163
[14]	validation-rmse:5.82336
[15]	validation-rmse:5.78348
[16]	validation-rmse:5.75276
[17]	validation-rmse:5.72899
[18]	validation-rmse:5.70985
[19]	validation-rmse:5.69499
[20]	validation-rmse:5.68360
[21]	validation-rmse:5.67469
[22]	validation-rmse:5.66855
[23]	validation-rmse:5.66246
[24]	validation-rmse:5.65777
[25]	validation-rmse:5.65464
[26]	validation-rmse:5.65224
[27]	validation-rmse:5.65069
[28]	validation-rmse:5.64943
[29]	validation-rmse:5.64841
[30]	validation-rmse:5.64776
[31]	validation-rmse:5.64629
[32]	validation-rmse:5.64368
[33]	validation-rmse:5.64300
[34]	validation-rmse:5.6